In [9]:
# Preparations
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import seaborn as sns
from IPython.display import Latex
import warnings
from PrettyTable import PrettyTable
from functools import partial
from PrettyFigure import PrettyFigure
warnings.filterwarnings("ignore", module="matplotlib")
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['savefig.dpi'] = 75

# plt.rcParams['figure.autolayout'] = False
# plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.unicode'] = True
plt.rcParams['font.family'] = "STIX"
plt.rcParams['text.latex.preamble'] = "\\usepackage{subdepth}, \\usepackage{type1cm}"

results = {}

sns.set(color_codes=True)

def average(data):
    return 1 / len(data) * sum(data)

def error(data, average_of_data):
    s = sum([(x - average_of_data)**2 for x in data])
    return math.sqrt(s / (len(data) * (len(data) - 1)))

def std_deviation(error_of_average, length_of_dataset):
    return error_of_average * math.sqrt(length_of_dataset)

def average_with_weights(data, weights):
    d = data
    w = weights
    return (d * w**-2).sum() / (w**-2).sum()

def error_with_weights(weights):
    w = weights
    return 1 / math.sqrt((w**-2).sum())

def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    return (d * w**-2).sum() / (w**-2).sum()

def werr(group, weight_name):
    return 1 / math.sqrt((group[weight_name]**-2).sum())

In [45]:
# Evaluate Data

s_s = 50e-6

s_s = 250e-6

# Read Data
df, slope, intercept = ({} for x in range(3))
dfb = pd.read_csv('data/laufzeitmethode.csv')
ax = None
for gas1 in ['luft']:
    df[gas1] = dfb.loc[dfb['gas1'] == gas1].loc[dfb['gas2'] == 'luft'].loc[dfb['p'] == 1]
    slope[gas1], intercept[gas1], sem, r, p = stats.linregress(df[gas1]['t'], df[gas1]['s'])
    n = np.linspace(0.3, 0.9, 100)
    print(df[gas1])

    ax = df[gas1].plot(kind='scatter', x='s', y='t', label='gemessene Laufzeit')
    plt.plot(n, [i * slope[gas1] + intercept[gas1] for i in n], label='linearer Fit der Laufzeit', axes=ax)
    plt.xlabel('Strecke [m]')
    plt.ylabel('Laufzeit [s]')
    plt.legend(bbox_to_anchor=(0.02, 0.98), loc=2, borderaxespad=0.2)
    plt.close()
plt.show()

   gas1  gas2    p    s   t    f     T
0  luft  luft  1.0  0.1   3  100  21.3
1  luft  luft  1.0  0.4  12  100  21.3
2  luft  luft  1.0  0.8  24  100  21.3
